## Image Workflow





In [2]:
# Libraries and variables from initial test
# Be sure to instal the azure-storage-blob (add to the requirements.txt document)
# !pip install azure-storage-blob
## I'm not entirely sure we need this, but it's here
# !pip install azure-identity
# !pip install -U python-dotenv




## STEP 1 - Test to make sure Blob Storage library is working

In [17]:
# Be sure we can load blob storage
# Load the libraries
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
    # Quick start code goes here
except Exception as ex:
    print('Exception:')
    print(ex)

Azure Blob Storage v12.7.1 - Python quickstart sample


## Step 2 - Add the Blob Storage Connection String

In [18]:
%load_ext dotenv
%dotenv
# settings.py
import os
# The following are a SAS connection string for an Azure storage account using Blob storage
# And a IoT Device SAS connection string for Azure IoT Hub.

BLOB_CON_STRING = os.getenv("CON_STRING")
HUB_CON_STRING = os.getenv("HUB_STRING")


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


## Step 3- Accepts the processing image from the AudioToSpectrum ControllerModule

In [19]:
# Setup - we need to move image files from our local data to the AudioToSpectrogram directory
# get the file path
path = '.\\testimages'
for files in os.listdir(path):
    if os.path.isfile(os.path.join(path, files)):
        print(files)

nn05e_20180504_000000 figure_2.png
nn06a_20180716_000000 figure_106.png
nn06a_20180716_000000 figure_13.png
nn06a_20180716_000000 figure_149.png
nn06a_20180716_000000 figure_151.png
nn06a_20180716_000000 figure_27.png
nn06a_20180716_000000 figure_28.png
nn06a_20180716_000000 figure_3.png
nn06a_20180716_000000 figure_34.png
nn06e_20180512_000000 figure_3.png


## Step 4 - Mock Audio Spectrogram Provider
* As a set up, we need to have the audiotospectrum container set up
* We also need to upload our testing images to it


In [20]:
# This will represent the AudioToSpectrogram MOCK process
import random

# 1. Get the files from the audiotospectrum folder in blob storage
# Create the inbox if it doesn't exists
# Create the BlobServiceClient object which will be used to create a container client
a2s_service_client = BlobServiceClient.from_connection_string(BLOB_CON_STRING)

# Create a unique name for the container
a2s_container_name = 'audiotospectrum'
inbox_container_name = 'inbox2'
account_name = 'sapg15g2'

# Create the container
a2s_container_client = a2s_service_client.get_container_client(a2s_container_name)
blobs = list(a2s_container_client.list_blobs())
blob = random.choice(blobs)
# now that we have a random image, we want to move it into the inbox

print(blob.name)

# Source
source_container_name = a2s_container_name
source_file_path = blob.name
source_blob = (f"https://{account_name}.blob.core.windows.net/{source_container_name}/{source_file_path}")

# Target
target_container_name = inbox_container_name
target_file_path = blob.name
copied_blob = a2s_service_client.get_blob_client(target_container_name, target_file_path)
copied_blob.start_copy_from_url(source_blob)


nn06a_20180716_000000 figure_34.png


{'etag': '"0x8D8D84B8BC59E2B"',
 'last_modified': datetime.datetime(2021, 2, 23, 22, 37, 5, tzinfo=datetime.timezone.utc),
 'client_request_id': 'a6f01e8e-7627-11eb-825a-38baf82f79bf',
 'request_id': '56677a74-601e-002c-2834-0a192c000000',
 'version': '2020-04-08',
 'version_id': None,
 'date': datetime.datetime(2021, 2, 23, 22, 37, 5, tzinfo=datetime.timezone.utc),
 'copy_id': '25cfc4c3-ebb6-460f-968a-24f697b94eb0',
 'copy_status': 'success'}

## Step 5 - Now time to pick up the file from the inbox

In [21]:
# Accepts the processing image from the AudioToSpectrogram ControllerModule

# this will probably need to go into the listen method on the message client,
# but for now, I'll just pull it from the inbox
inbox_service_client = BlobServiceClient.from_connection_string(BLOB_CON_STRING)
inbox_container_name = 'inbox'
blob_name='nn06a_20180716_000000 figure_28.png'
inbox_blob_client = inbox_service_client.get_blob_client(container=a2s_container_name, blob=blob_name)
# convert this to a byte image
byte_image = inbox_blob_client.download_blob().readall()


## Step 6 - post the image to the model on served on a docker instance

In [22]:
import requests

# If this fails, make sure the docker container is running locally
headers = {'Content-Type': 'application/octet-stream'}
response = requests.post('http://127.0.0.1/image', headers = headers, data = byte_image)

response.json()

{'created': '2021-02-23T22:38:28.449273',
 'id': '',
 'iteration': '',
 'predictions': [{'boundingBox': {'height': 0.14956676,
    'left': 0.1058976,
    'top': 0.60955017,
    'width': 0.67933979},
   'probability': 0.58929265,
   'tagId': 0,
   'tagName': 'rumble'},
  {'boundingBox': {'height': 0.12694419,
    'left': 0.18778198,
    'top': 0.32857194,
    'width': 0.40464241},
   'probability': 0.32663706,
   'tagId': 0,
   'tagName': 'rumble'},
  {'boundingBox': {'height': 0.12706349,
    'left': 0.23507464,
    'top': 0.55628984,
    'width': 0.52045192},
   'probability': 0.24418022,
   'tagId': 0,
   'tagName': 'rumble'},
  {'boundingBox': {'height': 0.15271979,
    'left': 0.19092798,
    'top': 0.6689933,
    'width': 0.51095524},
   'probability': 0.15568013,
   'tagId': 0,
   'tagName': 'rumble'},
  {'boundingBox': {'height': 0.18655946,
    'left': 0.32361362,
    'top': 0.30118832,
    'width': 0.44842984},
   'probability': 0.10838772,
   'tagId': 0,
   'tagName': 'rumble

## Step 7 - Install and use OpenCV to apply bounding box

In [9]:
! pip install opencv-python

In [10]:
import cv2
import numpy as np

In [23]:
# we need to write the file to the directory to use
# cv2, so we will do that here
## Download blobs
local_path = './data'
download_file_path = os.path.join(local_path, blob_name)
print('\nDownloading blob to \n\t' + download_file_path)

with open(download_file_path, 'wb') as download_file:
    download_file.write(byte_image)



	./data\nn06a_20180716_000000 figure_28.png


In [24]:
image = cv2.imread(download_file_path)
(h, w) = image.shape[:2]
print(h,w)

840 1120


## Step 8 - Get the predictions data for IoT Hub

In [25]:

predictions = response.json()['predictions']

In [14]:
#!pip install azure-iot-device

In [26]:
# can we write these values to the IoT hub
from azure.iot.device.aio import IoTHubDeviceClient

# Fetch the connection string from an enviornment variable
conn_str = (HUB_CON_STRING)

# Create instance of the device client using the authentication provider
device_client = IoTHubDeviceClient.create_from_connection_string(conn_str)

# Connect the device client.
await device_client.connect()

# Send a single message
print("Sending predictions...")
await device_client.send_message(str(response.json()))
print("Predictions successfully sent!")

# finally, disconnect
await device_client.disconnect()

Sending predictions...
Predictions successfully sent!


## Step 9 - Create the spectogram image with the bounding box and label

In [27]:
threshold = 30
color=(0,255,0)
for prediction in predictions:
    
    if prediction['probability']*100 > threshold and prediction['boundingBox']['left'] > 0.1 and prediction['boundingBox']['left']+prediction['boundingBox']['width']> 0.4:
        leftbox = int(w * prediction['boundingBox']['left'])
        topbox = int(h * prediction['boundingBox']['top'])
        widthbox = int(w * prediction['boundingBox']['width'])
        heightbox = int(h * prediction['boundingBox']['height'])
        
        cv2.putText(image, 'rumble '+str(prediction['probability']), (leftbox, topbox - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
       
        cv2.rectangle(image, (leftbox, topbox), (leftbox+widthbox, topbox+heightbox) ,color, 2)
    #cv2.imshow("output",image)
    cv2.imwrite('./data/posttest6.png',image)